# load_channels_snapshot_to_bigquery

### 🎯Objetivo

Este notebook no transforma datos.
Este notebook tiene una única responsabilidad:
Cargar el DataFrame df_channels_snapshot (generado en el notebook 03) hacia BigQuery como tabla histórica particionada.

Destino:
youtube-datasets-360.angelgarciadatablog.channels_snapshot

In [ ]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "channels_snapshot"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)

## 🧱 Cargar snapshot desde Parquet (temporal)  

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

file_path = PROCESSED_PATH / "channels_snapshot.parquet"

print("Ruta:", file_path)
print("Existe:", file_path.exists())

df_channels_snapshot = pd.read_parquet(file_path)

df_channels_snapshot.head()



In [ ]:
df_channels_snapshot.dtypes

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 

In [ ]:
#Crear la tabla solo la primera vez, luego ya no se creará

from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("snapshot_date", "DATE"),
    SchemaField("channel_id", "STRING"),
    SchemaField("subscriber_count", "INT64"),
    SchemaField("view_count", "INT64"),
    SchemaField("video_count", "INT64"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("Tabla ya existe.")
    
except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)

    table.time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="snapshot_date",
    )

    client.create_table(table)
    print("Tabla creada con partición.")



## 🔒 Control de idempotencia por `snapshot_date`

Antes de insertar el snapshot actual, se eliminan los registros existentes con el mismo `snapshot_date`.

Esto garantiza que el proceso sea **idempotente**:  Si el pipeline se ejecuta múltiples veces el mismo día (pruebas, re-ejecuciones o errores), el resultado final en BigQuery será siempre consistente y sin duplicados.


In [ ]:
#Eliminar datos si tienen snapshot_date del mismo día
snapshot_date = df_channels_snapshot["snapshot_date"].iloc[0]

delete_query = f"""
DELETE FROM `{FULL_TABLE_ID}`
WHERE snapshot_date = @snapshot_date
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter(
            "snapshot_date",
            "DATE",
            snapshot_date
        )
    ]
)

client.query(delete_query, job_config=job_config).result()

print(f"Snapshots del {snapshot_date} eliminados si existían.")



## 📌 Cargar datos del parquet a big query

In [ ]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE APPEND = Agrega fila nuevas
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df_channels_snapshot,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Carga completada correctamente.")